In [1]:
import os
import pandas as pd
from pathlib import Path
import shutil
import numpy as np
import random
import time
import gc
from typing import List, Tuple, Optional
import math
import matplotlib.pyplot as plt
from ultralytics import YOLO  
import torch
from PIL import Image


In [2]:

def split_data(images_dir, labels_dir, train_images_dir, train_labels_dir, val_images_dir, val_labels_dir, split_ratio=0.8):
    """
    Разделяет данные на обучающую и валидационную выборки.
    
    Args:
        images_dir (Path): Путь к корневой директории с изображениями (processed_images/train/).
        labels_dir (Path): Путь к корневой директории с аннотациями YOLO (processed_labels/train/).
        train_images_dir (Path): Путь к директории для обучающих изображений.
        train_labels_dir (Path): Путь к директории для обучающих аннотаций.
        val_images_dir (Path): Путь к директории для валидационных изображений.
        val_labels_dir (Path): Путь к директории для валидационных аннотаций.
        split_ratio (float, optional): Доля данных для обучения.
    """
    # Собираем все изображения и аннотации
    image_files = list(images_dir.glob('*.jpg'))
    label_files = list(labels_dir.glob('*.txt'))
    
    print(f"Найдено {len(image_files)} изображений и {len(label_files)} аннотаций.")
    
    # Создаём словари для сопоставления изображений и аннотаций по имени файла
    image_dict = {img.stem: img for img in image_files}
    label_dict = {lbl.stem: lbl for lbl in label_files}
    
    # Нахождение общих ключей (имена файлов без расширений)
    common_keys = set(image_dict.keys()).intersection(set(label_dict.keys()))
    
    print(f"Найдено {len(common_keys)} общих пар изображений и аннотаций.")
    
    if len(common_keys) == 0:
        print("Нет общих пар изображений и аннотаций. Проверьте, правильно ли выполнена конвертация аннотаций.")
        return
    
    # Создаём списки общих изображений и аннотаций
    common_images = [image_dict[k] for k in common_keys]
    common_labels = [label_dict[k] for k in common_keys]
    
    # Перемешиваем данные
    combined = list(zip(common_images, common_labels))
    random.shuffle(combined)
    common_images[:], common_labels[:] = zip(*combined)
    
    # Определяем индекс разделения
    split_idx = int(len(common_images) * split_ratio)
    train_images = common_images[:split_idx]
    train_labels = common_labels[:split_idx]
    val_images = common_images[split_idx:]
    val_labels = common_labels[split_idx:]
    
    print(f"Копирование {len(train_images)} изображений и аннотаций в обучающую выборку.")
    print(f"Копирование {len(val_images)} изображений и аннотаций в валидационную выборку.")
    
    # Копируем обучающие данные
    for img, lbl in zip(train_images, train_labels):
        shutil.copy(img, train_images_dir / img.name)
        shutil.copy(lbl, train_labels_dir / lbl.name)
    
    # Копируем валидационные данные
    for img, lbl in zip(val_images, val_labels):
        shutil.copy(img, val_images_dir / img.name)
        shutil.copy(lbl, val_labels_dir / lbl.name)
    
    print("Разделение данных завершено.")


In [3]:

class Metrics:
    @staticmethod
    def position_error(label_gt: np.ndarray, label_pr: np.ndarray, step=8, alpha=1.5, e1=5, e2=5) -> float:
        # Коды gt:
        # 0 - мяч не в кадре
        # 1 - мяч легко идентифицируем
        # 2 - мяч в кадре, но не легко идентифицируем
        # 3 - мяч скрыт
        if label_gt[0] != 0 and label_pr[0] == 0:
            return e1
        if label_gt[0] == 0 and label_pr[0] != 0:
            return e2
        dist = math.sqrt((label_gt[1] - label_pr[1]) ** 2 + (label_gt[2] - label_pr[2]) ** 2)
        pe = math.floor(dist / step) ** alpha
        pe = min(pe, 5)
        return pe

    @staticmethod
    def evaluate_predictions(labels_gt: np.ndarray, labels_pr: np.ndarray) -> Tuple[List[float], float]:
        pe = [Metrics.position_error(labels_gt[i], labels_pr[i]) for i in range(len(labels_gt))]
        SIBATRACC_vals = []
        for i in range(len(pe)):
            cumulative_pe = sum(pe[:i + 1])
            SIBATRACC_vals.append(1 - cumulative_pe / ((i + 1) * 5))
        SIBATRACC_total = 1 - sum(pe) / (len(labels_gt) * 5) if len(labels_gt) > 0 else 0
        return SIBATRACC_vals, SIBATRACC_total


In [14]:
import gc
from typing import List
from pathlib import Path
import gdown

def get_game_clip_pairs(data_path: Path, games: List[int]) -> List[tuple]:
    image_dir = data_path / "images" / "test"
    clip_names = set()
    for image_file in image_dir.glob("*.jpg"):
        parts = image_file.name.split('_')
        clip_name = '_'.join(parts[:-1])
        clip_names.add(clip_name)

    game_clip_pairs = []
    for clip_name in sorted(clip_names):
        g_part = clip_name.split('_')[0]  
        g_num = int(g_part.replace('game', ''))
        if g_num in games:
            game_clip_pairs.append((g_num, clip_name))
            
    return game_clip_pairs


class CustomTrackingModel:
    def __init__(self, batch_size: int, stack_size: int, model_path: Path, data_path: Path, output_path: Path):
        self.batch_size = batch_size
        self.stack_size = stack_size
        self.model_path = model_path
        self.data_path = data_path
        self.output_path = output_path
        self.model = None  # Будет загружена в методе load()

    def load(self, path="best.pt", load_from_cloud=True):
        # LBL5
        print("Загрузка модели YOLOv11...")
        if load_from_cloud:
            url = f"https://drive.google.com/uc?id=1Wegd5CSWgk6iKoyDNmex6gNwhUmkhSQI"
            print(f"Loading model from {url}")
            gdown.download(url, path, quiet=False)
            print(f"Loading model from {path}")
            self.model = YOLO(str(path), task='detect')

        else:
            self.model = YOLO(str(self.model_path))
        print("Модель YOLOv11 успешно загружена.")

    def load_images(self, clip_name: str) -> np.ndarray:
        image_dir = self.data_path / "images" / "test"
        images = []
        for image_file in sorted(image_dir.glob(f"{clip_name}*.jpg")):
            img = Image.open(image_file).convert("RGB")
            images.append(np.array(img))
        return np.array(images)

    def load_labels(self, clip_name: str) -> np.ndarray:
        label_dir = self.data_path / "labels" / "test"
        labels = []
        image_width = 1280
        image_height = 720

        for label_file in sorted(label_dir.glob(f"{clip_name}*.txt")):
            with open(label_file, 'r') as f:
                lines = f.readlines()
                if len(lines) == 0:
                    labels.append([0.0, 0.0, 0.0])
                else:
                    class_id, x_center, y_center, width, height = map(float, lines[0].split())
                    x_center_abs = x_center * image_width
                    y_center_abs = y_center * image_height
                    labels.append([class_id, x_center_abs, y_center_abs])
        
        return np.array(labels)
    
    def convert_labels_to_yolo(self, game, clip, image_dir, label_csv, output_images_dir, output_labels_dir, image_width=1280, image_height=720):
        #LBL4
        """
        Конвертирует аннотации из labels.csv в формат YOLO с уникальными именами файлов.
        Копирует изображения в единую директорию с уникальными именами.
        Записывает все аннотации для одного изображения в один .txt файл.
        
        Args:
            game (str): Название игры (например, 'game1').
            clip (str): Название клипа (например, 'clip1').
            image_dir (Path): Путь к директории с изображениями.
            label_csv (Path): Путь к файлу labels.csv.
            output_images_dir (Path): Путь к директории для сохранения изображений с уникальными именами.
            output_labels_dir (Path): Путь к директории для сохранения YOLO-аннотаций.
            image_width (int): Ширина изображений.
            image_height (int): Высота изображений.
        """
        output_images_dir.mkdir(parents=True, exist_ok=True)
        output_labels_dir.mkdir(parents=True, exist_ok=True)
        
        if not label_csv.exists():
            print(f"Файл {label_csv} не найден. Пропуск...")
            return
        
        df = pd.read_csv(label_csv)
        
        grouped = df.groupby('file name')
        
        for filename, group in grouped:
            visibility = group['visibility'].iloc[0] 
            if visibility == 0:

                unique_stem = f"{game}_{clip}_{Path(filename).stem}"
                label_path = output_labels_dir / f"{unique_stem}.txt"
                label_path.touch(exist_ok=True)
                
                image_path = image_dir / filename
                unique_image_name = f"{unique_stem}.jpg"
                new_image_path = output_images_dir / unique_image_name
                if not new_image_path.exists():
                    shutil.copy(image_path, new_image_path)
                continue
            
            yolo_annotations = []
            
            for _, row in group.iterrows():
                x = row['x-coordinate']
                y = row['y-coordinate']
                status = row['status']
                visibility = row['visibility']

                if pd.isna(x) or pd.isna(y):
                    continue

                x_center = float(x) / image_width
                y_center = float(y) / image_height

                width = 0.05
                height = 0.05

                
                class_id = int(visibility)

                yolo_annotation = f"{class_id} {x_center} {y_center} {width} {height}"
                yolo_annotations.append(yolo_annotation)
            
            if not yolo_annotations:
                unique_stem = f"{game}_{clip}_{Path(filename).stem}"
                label_path = output_labels_dir / f"{unique_stem}.txt"
                label_path.touch(exist_ok=True)
            else:
                unique_stem = f"{game}_{clip}_{Path(filename).stem}"
                
                label_path = output_labels_dir / f"{unique_stem}.txt"
                with open(label_path, 'w') as f:
                    f.write('\n'.join(yolo_annotations) + '\n')
            
            image_path = image_dir / filename
            unique_image_name = f"{unique_stem}.jpg"
            new_image_path = output_images_dir / unique_image_name
            if not new_image_path.exists():
                shutil.copy(image_path, new_image_path)

    def predict_on_batch(self, batch: np.ndarray) -> List[dict]:
        if self.model is None:
            raise ValueError("Модель не загружена. Вызовите метод load() перед предсказанием.")
        image_list = [Image.fromarray(img) for img in batch.astype(np.uint8)]
        results = self.model(image_list, verbose=False)
        return results

    def get_labels_from_prediction(self, predictions: List[Optional[dict]], upscale_coords: bool, original_size: Tuple[int, int]) -> np.ndarray:
        coords = np.zeros((len(predictions), 7), dtype=np.float32) 
        original_width, original_height = original_size
        
        for i, frame_preds in enumerate(predictions):
            if frame_preds is None or not frame_preds.boxes:
                coords[i] = [0, 0, 0, 0, 0, 0, 0]
                continue
            ball_pred = max(frame_preds.boxes, key=lambda box: box.conf)
            class_id = int(ball_pred.cls)
            x1 = float(ball_pred.xyxy[0][0])
            y1 = float(ball_pred.xyxy[0][1])
            x2 = float(ball_pred.xyxy[0][2])
            y2 = float(ball_pred.xyxy[0][3])
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2
            if upscale_coords:
                scale_x = original_width / frame_preds.orig_shape[1]
                scale_y = original_height / frame_preds.orig_shape[0]
                x_center *= scale_x
                y_center *= scale_y
                x1 *= scale_x
                y1 *= scale_y
                x2 *= scale_x
                y2 *= scale_y
            coords[i] = [class_id, x_center, y_center, x1, y1, x2, y2]
        
        return coords

    def evaluate_predictions(self, ground_truth: np.ndarray, predictions: np.ndarray) -> float:
        SIBATRACC_per_frame, SIBATRACC_total = Metrics.evaluate_predictions(ground_truth, predictions)
        return SIBATRACC_total

    def test(self, data_path: Path, games: List[int], do_visualization=False, test_name='test', yolo_format=False) -> float:
        if not yolo_format:
            test_games = games
            test_dir = data_path / test_name

            output_images_dir = Path('processed_images/test/')
            output_labels_dir = Path('processed_labels/test/')

            for game in test_games:
                game_dir = test_dir / f'game{game}'
                if not game_dir.exists():
                    print(f"Директория {game_dir} не найдена. Пропуск...")
                    continue
                for clip in game_dir.iterdir():
                    if clip.is_dir():
                        image_dir = clip
                        label_csv = clip / 'labels.csv'
                        if label_csv.exists():
                            self.convert_labels_to_yolo(
                                game=f'game{game}',
                                clip=clip.name,
                                image_dir=image_dir,
                                label_csv=label_csv,
                                output_images_dir=output_images_dir,
                                output_labels_dir=output_labels_dir,
                                image_width=1280,
                                image_height=720
                            )
                        else:
                            print(f"Файл {label_csv} не найден в {clip}. Пропуск...")

            test_images_dir = Path('dataset/images/test/')
            test_labels_dir = Path('dataset/labels/test/')
            test_images_dir.mkdir(parents=True, exist_ok=True)
            test_labels_dir.mkdir(parents=True, exist_ok=True)

            split_data(
                images_dir=output_images_dir,
                labels_dir=output_labels_dir,
                train_images_dir=test_images_dir,
                train_labels_dir=test_labels_dir,
                val_images_dir=None,
                val_labels_dir=None,
                split_ratio=1.0
            )

            
        game_clip_pairs = get_game_clip_pairs(self.data_path, games)
        SIBATRACC_vals = []
        for game, clip_name in game_clip_pairs:
            data = self.load_images(clip_name)
            labels_gt = self.load_labels(clip_name)

            predictions = self.predict_on_batch(data)
            predicted_labels = self.get_labels_from_prediction(predictions, upscale_coords=True, original_size=(1280, 720))

            SIBATRACC_per_frame, SIBATRACC_total = Metrics.evaluate_predictions(labels_gt, predicted_labels)
            SIBATRACC_vals.append(SIBATRACC_total)

            if do_visualization:
                self.visualize_predictions(data, predicted_labels, clip_name)

            del data, labels_gt, predicted_labels, predictions
            gc.collect()

        SIBATRACC_final = sum(SIBATRACC_vals) / len(SIBATRACC_vals) if SIBATRACC_vals else 0
        return SIBATRACC_final

    def visualize_predictions(self, images: np.ndarray, predicted_labels: np.ndarray, clip_name: str):
        visualize_path = self.output_path / "visualizations"
        visualize_path.mkdir(parents=True, exist_ok=True)

        for i, img in enumerate(images):
            plt.figure(figsize=(10, 6))
            plt.imshow(img)
            plt.axis('off')
            
            class_id, x_center, y_center, x1, y1, x2, y2 = predicted_labels[i]

            if class_id != 0:
                rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2)
                plt.gca().add_patch(rect)
                plt.scatter(x_center, y_center, c='blue', s=20, marker='x')
                plt.text(x1, y1 - 5, f'Class: {class_id}', color='yellow', fontsize=12, backgroundcolor='black')
            
            plt.title(f'{clip_name} Frame {i}')
            save_path = visualize_path / f"{clip_name}_frame_{i}.jpg"
            plt.savefig(save_path, bbox_inches='tight')
            plt.close()


    def train_model(self, data_config: Path, epochs: int = 50, imgsz: int = 640, load_existing: bool = True, yolo_format: bool = False):
        # LBL1, LBL2, LBL3, LBL6, LBL13, LBL7
        if not yolo_format:
            train_games = [1, 2, 3, 4, 5, 6]  # Список игр для обучения
            train_dir = Path('tennis/train/')  # Путь к тренировочным данным
            yolo_labels_dir = Path('yolo_labels/train/')  # Путь для сохранения YOLO-аннотаций

            # Путь к целевым директориям с уникальными именами
            output_images_dir = Path('processed_images/train/')
            output_labels_dir = Path('processed_labels/train/')

            for game in train_games:
                game_dir = train_dir / f'game{game}'
                if not game_dir.exists():
                    print(f"Директория {game_dir} не найдена. Пропуск...")
                    continue
                for clip in game_dir.iterdir():
                    if clip.is_dir():
                        image_dir = clip
                        label_csv = clip / 'labels.csv'
                        if label_csv.exists():
                            self.convert_labels_to_yolo(
                                game=f'game{game}',
                                clip=clip.name,
                                image_dir=image_dir,
                                label_csv=label_csv,
                                output_images_dir=output_images_dir,
                                output_labels_dir=output_labels_dir,
                                image_width=1280,
                                image_height=720
                            )
                        else:
                            print(f"Файл {label_csv} не найден в {clip}. Пропуск...")
            train_images_dir = Path('dataset/images/train/')
            train_labels_dir = Path('dataset/labels/train/')
            val_images_dir = Path('dataset/images/val/')
            val_labels_dir = Path('dataset/labels/val/')

            train_images_dir.mkdir(parents=True, exist_ok=True)
            train_labels_dir.mkdir(parents=True, exist_ok=True)
            val_images_dir.mkdir(parents=True, exist_ok=True)
            val_labels_dir.mkdir(parents=True, exist_ok=True)

            processed_images_dir = Path('processed_images/train/')
            processed_labels_dir = Path('processed_labels/train/')

            split_data(
                images_dir=processed_images_dir,
                labels_dir=processed_labels_dir,
                train_images_dir=train_images_dir,
                train_labels_dir=train_labels_dir,
                val_images_dir=val_images_dir,
                val_labels_dir=val_labels_dir,
                split_ratio=0.8
            )
        if self.model is None:
            if self.model_path.exists() and load_existing:
                self.model = YOLO(str(self.model_path))
            else:
                self.model = YOLO('yolo11l.pt')  

        self.model.train(
            data=str(data_config),
            epochs=epochs,
            batch=self.batch_size,
            imgsz=imgsz,
            workers=32,
            device='0'
        )

    def predict_with_tracking(self, clip_name: str, image_dir: Path) -> List[dict]:
        if self.model is None:
            raise ValueError("Модель не загружена. Вызовите метод load() перед предсказанием.")

        results = self.model.track(source=str(image_dir), conf=0.5, save=False, verbose=False, iou=0.5)
        all_predictions = []
        for res in results:
            frame_data = []
            for box in res.boxes:
                cls = int(box.cls)
                conf = float(box.conf)
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                track_id = int(box.id) if box.id is not None else -1
                x_center = (x1 + x2) / 2.0
                y_center = (y1 + y2) / 2.0
                frame_data.append({
                    'class_id': cls,
                    'conf': conf,
                    'track_id': track_id,
                    'x_center': x_center,
                    'y_center': y_center,
                    'x1': x1,
                    'y1': y1,
                    'x2': x2,
                    'y2': y2
                })
            all_predictions.append(frame_data)

        return all_predictions


In [8]:
model = CustomTrackingModel(
    batch_size=70,
    stack_size=4,
    model_path=Path(""),
    data_path=Path("dataset"),
    output_path=Path("output")
)

# Обучение модели (предполагается, что data.yaml настроен)
model.train_model(data_config=Path("dataset.yaml"), epochs=50, imgsz=640, load_existing=False)

New https://pypi.org/project/ultralytics/8.3.51 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.47 🚀 Python-3.11.5 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-80GB, 81158MiB)
                                                      CUDA:1 (NVIDIA A100-SXM4-80GB, 81158MiB)
                                                      CUDA:3 (NVIDIA A100-SXM4-80GB, 81158MiB)
                                                      CUDA:4 (NVIDIA A100-SXM4-80GB, 81158MiB)
                                                      CUDA:5 (NVIDIA A100-SXM4-80GB, 81158MiB)
                                                      CUDA:6 (NVIDIA A100-SXM4-80GB, 81158MiB)
                                                      CUDA:7 (NVIDIA A100-SXM4-80GB, 81158MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=dataset.yaml, epochs=50, time=None, patience=100, batch=70, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 3, 4, 5, 6, 7], workers=32, project=None, name

E0000 00:00:1734640904.815290 3999282 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734640904.820624 3999282 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  7                  -1  1   2360320  ultralytics

E0000 00:00:1734640914.896321 3999908 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734640914.901641 3999908 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorBoard: Start with 'tensorboard --logdir runs/detect/train10', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=4
Transferred 1009/1015 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


...

                   all       7982       7580      0.834      0.768      0.843      0.678

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      7.28G     0.5944      0.347     0.8041          1        640: 100%|██████████| 171/171 [00:47<00:00,  3.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 400/400 [00:31<00:00, 12.62it/s]


                   all       7982       7580      0.826      0.791       0.85      0.678

50 epochs completed in 1.117 hours.
Optimizer stripped from runs/detect/train10/weights/last.pt, 51.2MB
Optimizer stripped from runs/detect/train10/weights/best.pt, 51.2MB

Validating runs/detect/train10/weights/best.pt...
Ultralytics 8.3.47 🚀 Python-3.11.5 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-80GB, 81158MiB)
                                                      CUDA:1 (NVIDIA A100-SXM4-80GB, 81158MiB)
                                                      CUDA:3 (NVIDIA A100-SXM4-80GB, 81158MiB)
                                                      CUDA:4 (NVIDIA A100-SXM4-80GB, 81158MiB)
                                                      CUDA:5 (NVIDIA A100-SXM4-80GB, 81158MiB)
                                                      CUDA:6 (NVIDIA A100-SXM4-80GB, 81158MiB)
                                                      CUDA:7 (NVIDIA A100-SXM4-80GB, 81158MiB)
YOLO11l summary (fused)

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 400/400 [00:31<00:00, 12.67it/s]


                   all       7982       7580      0.826      0.791       0.85      0.679
          easy_visible       6911       6911       0.97      0.986      0.992      0.871
      visible_not_easy        626        626      0.729      0.736      0.805      0.638
                hidden         43         43      0.778      0.651      0.752      0.528
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train10


In [15]:
#  Пример использования:
model = CustomTrackingModel(
    batch_size=32,
    stack_size=4,
    model_path=Path("runs/detect/train10/weights/best.pt"),
    data_path=Path("dataset"),
    output_path=Path("output")
)

model.load()  # Загружаем модель


Загрузка модели YOLOv11...
Loading model from https://drive.google.com/uc?id=1Wegd5CSWgk6iKoyDNmex6gNwhUmkhSQI


Downloading...
From (original): https://drive.google.com/uc?id=1Wegd5CSWgk6iKoyDNmex6gNwhUmkhSQI
From (redirected): https://drive.google.com/uc?id=1Wegd5CSWgk6iKoyDNmex6gNwhUmkhSQI&confirm=t&uuid=27305e22-5d13-4032-8ed9-58408fec1807
To: /opt/notebooks/sports-tracking-tystem/best.pt
100%|██████████| 51.2M/51.2M [00:00<00:00, 105MB/s] 


Loading model from best.pt
Модель YOLOv11 успешно загружена.


In [16]:

average_sibatracc = model.test(data_path=Path("tennis"), games=[1, 2], do_visualization=False, test_name='test', yolo_format=False)
print(f"Среднее значение SIBATRACC: {average_sibatracc}")


Найдено 3795 изображений и 3795 аннотаций.
Найдено 3795 общих пар изображений и аннотаций.
Копирование 3795 изображений и аннотаций в обучающую выборку.
Копирование 0 изображений и аннотаций в валидационную выборку.
Разделение данных завершено.
Среднее значение SIBATRACC: 0.8580679450223773


In [14]:
def create_video_from_images_with_tracking(model, image_files: list, clip_name: str, output_path: Path, fps: int = 30):
    if not image_files:
        print(f"No images found for {clip_name}.")
        return
    print(f"Found {len(image_files)} images for {clip_name}. Creating tracked video...")

    temp_dir = Path("temp_tracking_dir")
    if temp_dir.exists():
        shutil.rmtree(temp_dir)
    temp_dir.mkdir(parents=True, exist_ok=True)

    for img_file in image_files:
        shutil.copy(img_file, temp_dir / img_file.name)

    results = model.predict_with_tracking(clip_name=clip_name, image_dir=temp_dir)

    first_frame = cv2.imread(str(image_files[0]))
    if first_frame is None:
        print("Unable to read the first frame. Check your images.")
        shutil.rmtree(temp_dir)
        return

    height, width, _ = first_frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

    for i, frame_results in enumerate(results):
        frame = cv2.imread(str(image_files[i]))
        if frame is None:
            print(f"Unable to read frame {image_files[i]}. Skipping...")
            continue
        for box in frame_results:
            x1, y1, x2, y2 = int(box['x1']), int(box['y1']), int(box['x2']), int(box['y2'])
            track_id = box['track_id']
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, max(0, y1 - 5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        video.write(frame)

    video.release()
    print(f"Video with tracking saved to {output_path}.")
    shutil.rmtree(temp_dir)


def create_video_for_game_clip(model, game_number: int, clip_number: int, data_path: Path, output_path: Path, fps: int = 30):
    images_dir = data_path / "images" / "test"
    clip_name = f"game{game_number}_clip{clip_number}"
    image_files = sorted(images_dir.glob(f"{clip_name}_*.jpg"))

    output_file = output_path / f"{clip_name}_tracked.mp4"
    create_video_from_images_with_tracking(model, image_files, clip_name, output_file, fps)


def create_video_for_game(model, game_number: int, data_path: Path, output_path: Path, fps: int = 30):
    images_dir = data_path / "images" / "test"
    game_prefix = f"game{game_number}_clip"
    image_files = sorted(images_dir.glob(f"{game_prefix}*.jpg"))

    clip_name = f"game{game_number}_full"
    output_file = output_path / f"{clip_name}_tracked.mp4"
    create_video_from_images_with_tracking(model, image_files, clip_name, output_file, fps)



In [16]:

model = CustomTrackingModel(
    batch_size=32,
    stack_size=4,
    model_path=Path("runs/detect/train10/weights/best.pt"),
    data_path=Path("dataset"),
    output_path=Path("output")
)
model.load()

create_video_for_game_clip(model, game_number=2, clip_number=3, data_path=Path("dataset"), output_path=Path("output"), fps=30)

create_video_for_game(model, game_number=2, data_path=Path("dataset"), output_path=Path("output"), fps=30)


Загрузка модели YOLOv8...
Модель YOLOv8 успешно загружена.
Found 359 images for game2_clip3. Creating tracked video...
Video with tracking saved to output/game2_clip3_tracked.mp4.
Found 1573 images for game2_full. Creating tracked video...

WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Video with tracking saved to output/game2_full_tracked.mp4.
